<a href="https://colab.research.google.com/github/vokiet2005/TTNT/blob/main/baituan3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import copy
import math
import random
import numpy

# Định nghĩa các trạng thái có thể của một ô trên bàn cờ Tic-Tac-Toe
X = "X"  # Người chơi X
O = "O"  # Người chơi O
EMPTY = None  # Ô trống

# Khởi tạo các biến người chơi (sẽ được thiết lập bằng đầu vào người dùng sau)
user = None
ai = None

def initial_state():
    """
    Trả về trạng thái bắt đầu của bàn cờ.
    Bàn cờ là một lưới 3x3, ban đầu tất cả các ô đều EMPTY.
    """
    return [[EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY]]

def player(board):
    """
    Trả về người chơi có lượt đi tiếp theo trên bàn cờ.
    Đếm số ô không trống. Nếu số ô lẻ, đó là lượt đi của AI (vì người dùng bắt đầu).
    Nếu số ô chẵn, đó là lượt đi của người dùng.
    """
    count = 0
    for row in board:
        for cell in row:
            if cell is not EMPTY:  # Nếu ô không trống, một nước đi đã được thực hiện
                count += 1

    # Nếu số nước đi lẻ, đó là lượt của AI.
    # Nếu số nước đi chẵn, đó là lượt của người dùng.
    # Giả sử 'user' luôn bắt đầu trò chơi.
    if count % 2 != 0: # Sau nước đi đầu tiên của người dùng, count là 1, AI chơi. Sau nước đi của AI, count là 2, người dùng chơi.
        return ai
    return user

def actions(board):
    """
    Trả về tập hợp tất cả các hành động có thể (i, j) có sẵn trên bàn cờ.
    Một hành động được biểu thị bằng một tuple (hàng, cột) của một ô trống.
    """
    res = set()
    board_len = len(board) # Đối với bàn cờ 3x3, board_len là 3
    for i in range(board_len):
        for j in range(board_len):
            if board[i][j] == EMPTY: # Nếu ô trống, đó là một nước đi có thể
                res.add((i, j))
    return res

def result(board, action):
    """
    Trả về bàn cờ là kết quả của việc thực hiện nước đi (i, j) trên bàn cờ.
    Hàm này không sửa đổi bàn cờ gốc; nó trả về một trạng thái bàn cờ mới.
    """
    curr_player = player(board) # Xác định lượt của ai
    result_board = copy.deepcopy(board) # Tạo một bản sao sâu để tránh sửa đổi bàn cờ gốc
    (i, j) = action # Giải nén hành động (hàng, cột)
    result_board[i][j] = curr_player # Đặt dấu của người chơi hiện tại lên bàn cờ
    return result_board

def get_horizontal_winner(board):
    """
    Kiểm tra người thắng theo chiều ngang.
    """
    winner_val = None
    board_len = len(board)
    for i in range(board_len): # Lặp qua từng hàng
        winner_val = board[i][0] # Giả sử ô đầu tiên trong hàng là dấu của người thắng
        if winner_val is EMPTY: # Nếu ô đầu tiên trống, chưa có người thắng nào trong hàng này
            continue
        for j in range(board_len): # Kiểm tra tất cả các ô trong hàng hiện tại
            if board[i][j] != winner_val: # Nếu bất kỳ ô nào không khớp với dấu của người thắng được giả định
                winner_val = None # Không có người thắng trong hàng này
                break
        if winner_val: # Nếu tìm thấy người thắng trong hàng này
            return winner_val
    return winner_val # Không tìm thấy người thắng theo chiều ngang

def get_vertical_winner(board):
    """
    Kiểm tra người thắng theo chiều dọc.
    """
    winner_val = None
    board_len = len(board)
    for i in range(board_len): # Lặp qua từng chỉ số cột
        winner_val = board[0][i] # Giả sử ô đầu tiên trong cột là dấu của người thắng
        if winner_val is EMPTY: # Nếu ô đầu tiên trống, chưa có người thắng nào trong cột này
            continue
        for j in range(board_len): # Kiểm tra tất cả các ô trong cột hiện tại
            if board[j][i] != winner_val: # Nếu bất kỳ ô nào không khớp với dấu của người thắng được giả định
                winner_val = None # Không có người thắng trong cột này
                break
        if winner_val: # Nếu tìm thấy người thắng trong cột này
            return winner_val
    return winner_val # Không tìm thấy người thắng theo chiều dọc

def get_diagonal_winner(board):
    """
    Kiểm tra người thắng theo đường chéo (cả đường chéo chính và đường chéo phụ).
    """
    winner_val = None
    board_len = len(board)

    # Kiểm tra đường chéo chính (trên cùng bên trái đến dưới cùng bên phải)
    winner_val = board[0][0] # Giả sử ô đầu tiên là dấu của người thắng
    if winner_val is not EMPTY: # Chỉ kiểm tra nếu ô đầu tiên không trống
        found_winner = True
        for i in range(board_len):
            if board[i][i] != winner_val: # Nếu bất kỳ ô nào không khớp
                found_winner = False
                break
        if found_winner: # Nếu tất cả các ô khớp
            return winner_val

    # Kiểm tra đường chéo phụ (trên cùng bên phải đến dưới cùng bên trái)
    winner_val = board[0][board_len - 1] # Giả sử ô đầu tiên là dấu của người thắng
    if winner_val is not EMPTY: # Chỉ kiểm tra nếu ô đầu tiên không trống
        found_winner = True
        for i in range(board_len):
            j = board_len - 1 - i # Tính chỉ số cột cho đường chéo phụ
            if board[i][j] != winner_val: # Nếu bất kỳ ô nào không khớp
                found_winner = False
                break
        if found_winner: # Nếu tất cả các ô khớp
            return winner_val

    return None # Không tìm thấy người thắng theo đường chéo

def winner(board):
    """
    Trả về người thắng cuộc, nếu có.
    Kết hợp kiểm tra người thắng theo chiều ngang, dọc và đường chéo.
    """
    # Sử dụng 'or' để trả về người thắng đầu tiên không phải None được tìm thấy
    winner_val = get_horizontal_winner(board) or \
                 get_vertical_winner(board) or \
                 get_diagonal_winner(board)
    return winner_val

def terminal(board):
    """
    Trả về True nếu trò chơi kết thúc (có người thắng hoặc hòa), False nếu ngược lại.
    """
    if winner(board) is not None: # Nếu có người thắng, trò chơi kết thúc
        return True

    # Kiểm tra hòa (không còn ô trống nào)
    for row in board:
        for cell in row:
            if cell == EMPTY: # Nếu bất kỳ ô nào trống, trò chơi chưa kết thúc
                return False
    return True # Không có người thắng và không còn ô trống nào có nghĩa là hòa, trò chơi kết thúc

def utility(board):
    """
    Trả về 1 nếu X thắng, -1 nếu O thắng, 0 nếu hòa.
    Điều này được sử dụng cho thuật toán minimax để đánh giá các trạng thái bàn cờ.
    """
    winner_val = winner(board)
    if winner_val == X:
        return 1  # X thắng
    elif winner_val == O:
        return -1 # O thắng
    return 0  # Hòa hoặc trò chơi chưa kết thúc

def maxValue(state):
    """
    Tìm giá trị tiện ích tối đa cho người chơi tối đa hóa (X).
    Được sử dụng trong thuật toán minimax.
    """
    if terminal(state): # Trường hợp cơ sở: nếu trò chơi kết thúc, trả về tiện ích của nó
        return utility(state)
    v = -math.inf # Khởi tạo v thành âm vô cùng
    for action in actions(state): # Lặp qua tất cả các nước đi có thể
        v = max(v, minValue(result(state, action))) # Gọi đệ quy minValue cho trạng thái tiếp theo
    return v

def minValue(state):
    """
    Tìm giá trị tiện ích tối thiểu cho người chơi tối thiểu hóa (O).
    Được sử dụng trong thuật toán minimax.
    """
    if terminal(state): # Trường hợp cơ sở: nếu trò chơi kết thúc, trả về tiện ích của nó
        return utility(state)
    v = math.inf # Khởi tạo v thành dương vô cùng
    for action in actions(state): # Lặp qua tất cả các nước đi có thể
        v = min(v, maxValue(result(state, action))) # Gọi đệ quy maxValue cho trạng thái tiếp theo
    return v

def minimax(board):
    """
    Trả về hành động tối ưu cho người chơi hiện tại trên bàn cờ bằng cách sử dụng thuật toán Minimax.
    Thuật toán khám phá tất cả các nước đi tương lai có thể để xác định nước đi tốt nhất tiếp theo.
    """
    current_player = player(board)

    if current_player == X:  # Người chơi tối đa hóa (X muốn tối đa hóa điểm số)
        min_val = -math.inf # Lưu trữ tiện ích tối đa được tìm thấy cho X cho đến nay
        move = None # Lưu trữ nước đi tốt nhất cho X
        for action in actions(board): # Xem xét từng hành động có thể
            # Đánh giá kết quả của hành động này giả sử cả hai bên đều chơi tối ưu
            check = minValue(result(board, action)) # Sau khi X chơi, đến lượt O (minValue)
            if check > min_val: # Nếu nước đi này dẫn đến kết quả tốt hơn cho X
                min_val = check # Cập nhật tiện ích tốt nhất
                move = action # Cập nhật nước đi tốt nhất
        return move # Trả về nước đi tối ưu cho X
    else:  # Người chơi tối thiểu hóa (O muốn tối thiểu hóa điểm số của X, thực tế là tối đa hóa điểm số của O)
        max_val = math.inf # Lưu trữ tiện ích tối thiểu được tìm thấy cho O cho đến nay
        move = None # Lưu trữ nước đi tốt nhất cho O
        for action in actions(board): # Xem xét từng hành động có thể
            # Đánh giá kết quả của hành động này giả sử cả hai bên đều chơi tối ưu
            check = maxValue(result(board, action)) # Sau khi O chơi, đến lượt X (maxValue)
            if check < max_val: # Nếu nước đi này dẫn đến kết quả tốt hơn cho O
                max_val = check # Cập nhật tiện ích tốt nhất
                move = action # Cập nhật nước đi tốt nhất
        return move # Trả về nước đi tối ưu cho O

# Vòng lặp chính của trò chơi
if __name__ == "__main__":
    board = initial_state() # Khởi tạo một bàn cờ Tic-Tac-Toe trống
    ai_turn = False # Cờ để theo dõi lượt của ai. Người dùng bắt đầu, vì vậy lượt của AI ban đầu là False.

    print("Chọn người chơi (X hoặc O):")
    user_input = input().upper() # Lấy lựa chọn của người dùng và chuyển đổi thành chữ hoa để nhất quán

    # Xác thực đầu vào cho lựa chọn của người dùng (X hoặc O)
    while user_input not in ["X", "O"]:
        print("Lựa chọn không hợp lệ. Vui lòng chọn 'X' hoặc 'O'.")
        user_input = input().upper()

    user = user_input
    if user == "X":
        ai = "O"
    else:
        ai = "X"

    print(f"Bạn là {user}. AI là {ai}.")

    while True:
        game_over = terminal(board) # Kiểm tra xem trò chơi đã kết thúc chưa
        playr = player(board) # Xác định lượt của ai bây giờ

        if game_over: # Nếu trò chơi kết thúc
            winner_val = winner(board) # Lấy người thắng cuộc (hoặc None nếu hòa)
            if winner_val is None:
                print("Trò chơi kết thúc: Hòa.")
            else:
                print(f"Trò chơi kết thúc: {winner_val} thắng.")
            break # Thoát vòng lặp trò chơi
        else:
            # Nếu là lượt của AI và trò chơi chưa kết thúc
            if playr == ai and not game_over:
                print(f"AI ({ai}) đang thực hiện nước đi...")
                move = minimax(board) # AI tính toán nước đi tối ưu
                board = result(board, move) # Cập nhật bàn cờ với nước đi của AI
                print("Nước đi của AI:")
                print(numpy.array(board)) # Hiển thị bàn cờ sau nước đi của AI

            # Nếu là lượt của người dùng và trò chơi chưa kết thúc
            elif playr == user and not game_over:
                print(f"Lượt của bạn ({user}).")
                print("Bàn cờ hiện tại:")
                print(numpy.array(board)) # Hiển thị bàn cờ hiện tại trước nước đi của người dùng

                valid_move = False
                while not valid_move:
                    try:
                        print("Nhập vị trí để di chuyển (hàng,cột), ví dụ: 0 cho hàng đầu tiên, 1 cho cột thứ hai:")
                        i = int(input("Hàng (0-2): "))
                        j = int(input("Cột (0-2): "))

                        # Xác thực đầu vào người dùng: trong giới hạn bàn cờ và ô trống
                        if 0 <= i < 3 and 0 <= j < 3 and board[i][j] == EMPTY:
                            board = result(board, (i, j)) # Cập nhật bàn cờ với nước đi của người dùng
                            valid_move = True # Nước đi hợp lệ, thoát vòng lặp nhập
                        else:
                            print("Nước đi không hợp lệ. Vui lòng chọn một ô trống trong giới hạn bàn cờ (0-2).")
                    except ValueError:
                        print("Đầu vào không hợp lệ. Vui lòng nhập số cho hàng và cột.")
